<a href="https://colab.research.google.com/github/CIS6930-NLP/final_project/blob/main/Finetune_DistilBERT_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Import libraries

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 100.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 17.9 MB/s eta 0:00:00


In [4]:
import tensorflow as tf
import re
import nltk
import numpy as np
import math
import time
import pandas as pd
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
# from transformers import BertTokenizer, TFBertForQuestionAnswering

print(tf.__version__)

2.11.0


In [5]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0



#### Loading data

In [7]:
#load data
train_file = "/content/drive/MyDrive/code/Updated Data/train.csv"
val_file = "/content/drive/MyDrive/code/Updated Data/valid.csv"
test_file = "/content/drive/MyDrive/code/Updated Data/test.csv"
def data_loading(filepath): 
  data = pd.read_csv(filepath, encoding = 'utf-8')
  data = data.drop('conv_id', axis = 1)
  data = data.drop('utterance_idx', axis = 1)
  data = data.drop('speaker_idx', axis = 1)
  data = data.drop('selfeval', axis = 1)
  data = data.drop('tags', axis = 1)
  data = data.dropna()
  return data

train_data = data_loading(train_file)
val_data = data_loading(val_file)
test_data = data_loading(test_file)
train_data.head()

,Unnamed: 0,context,prompt,utterance
0,0,sentimental,I remember going to the fireworks with my best...,I remember going to see the fireworks with my ...
1,1,sentimental,I remember going to the fireworks with my best...,Was this a friend you were in love with_comma_...
2,2,sentimental,I remember going to the fireworks with my best...,This was a best friend. I miss her.
3,3,sentimental,I remember going to the fireworks with my best...,Where has she gone?
4,4,sentimental,I remember going to the fireworks with my best...,We no longer talk.


In [8]:
#emotion preprocessing
#group emotions
emotions = {}
emotions['excited'] = emotions['surprised'] = emotions['joyful'] = "excited"
emotions['afraid'] = emotions['terrified'] = emotions['anxious']= emotions['apprehensive']='afraid'
emotions['disgusted'] = emotions['embarrassed']= emotions['guilty'] = emotions['ashamed'] ="disgusted"
emotions['angry'] = emotions ['annoyed'] = emotions['jealous'] =emotions[ 'furious' ] = "annoyed"
emotions['faithful'] = emotions ['trusting']=emotions ['grateful']= emotions['caring'] = emotions['hopeful'] = "grateful"
emotions['sad'] = emotions['disappointed'] = emotions['devastated']= emotions ['lonely']=emotions['nostalgic']=emotions['sentimental'] = "disappointed"
emotions['proud']= emotions['impressed']= emotions['content'] = "impressed"
emotions['anticipating']=emotions[ 'prepared']=emotions ['confident'] = "prepared"
dicttt=emotions

#### Data preprocessing

In [9]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r"i'm", "i am", sentence)
    sentence = re.sub(r"he's", "he is", sentence)
    sentence = re.sub(r"she's", "she is", sentence)
    sentence = re.sub(r"it's", "it is", sentence)
    sentence = re.sub(r"that's", "that is", sentence)
    sentence = re.sub(r"what's", "what is", sentence)
    sentence = re.sub(r"where's", "where is", sentence)
    sentence = re.sub(r"how's", "how is", sentence)
    sentence = re.sub(r"\'ll", " will", sentence)
    sentence = re.sub(r"\'ve", " have", sentence)
    sentence = re.sub(r"\'re", " are", sentence)
    sentence = re.sub(r"\'d", " would", sentence)
    sentence = re.sub(r"\'re", " are", sentence)
    sentence = re.sub(r"won't", "will not", sentence)
    sentence = re.sub(r"can't", "cannot", sentence)
    sentence = re.sub(r"n't", " not", sentence)
    sentence = re.sub(r"n'", "ng", sentence)
    sentence = re.sub(r"'bout", "about", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)
    sentence = sentence.strip()

    return sentence

In [32]:
train_context = train_data['context'][:5000]
train_question = train_data['prompt'][:5000]
train_answer = train_data['utterance'][:5000]


val_context = val_data['context'][:200]
val_question = val_data['prompt'][:200]
val_answer = val_data['utterance'][:200]

test_context = test_data['context'][:1000]
test_question = test_data['prompt'][:1000]
test_answer = test_data['utterance'][:1000]
# print(len(train_context))
# print(len(train_question))
# print(len(train_answer))

  # Maximum number of samples to preprocess
MAX_SAMPLES = 50000

train_context = [preprocess_sentence(emotions[sentence]) for sentence in train_context]
train_questions = [preprocess_sentence(sentence) for sentence in train_question]
train_answers = [preprocess_sentence(sentence) for sentence in train_answer]

val_context = [preprocess_sentence(emotions[sentence]) for sentence in val_context]
val_questions = [preprocess_sentence(sentence) for sentence in val_question]
val_answers = [preprocess_sentence(sentence) for sentence in val_answer]

test_context = [preprocess_sentence(emotions[sentence]) for sentence in test_context]
test_questions = [preprocess_sentence(sentence) for sentence in test_question]
test_answers = [preprocess_sentence(sentence) for sentence in test_answer]

train_data['context'] = [preprocess_sentence(emotions[sentence]) for sentence in train_data['context']]
train_data['prompt'] = [preprocess_sentence(sentence) for sentence in train_data['prompt']]
train_data['utterance'] = [preprocess_sentence(sentence) for sentence in train_data['utterance']]

val_data['context'] = [preprocess_sentence(emotions[sentence]) for sentence in val_data['context']]
val_data['prompt'] = [preprocess_sentence(sentence) for sentence in val_data['prompt']]
val_data['utterance'] = [preprocess_sentence(sentence) for sentence in val_data['utterance']]

test_data['context'] = [preprocess_sentence(emotions[sentence]) for sentence in test_data['context']]
test_data['prompt'] = [preprocess_sentence(sentence) for sentence in test_data['prompt']]
test_data['utterance'] = [preprocess_sentence(sentence) for sentence in test_data['utterance']]

In [11]:
print('Sample context: {}'.format(train_data['context'][20]))
print('Sample question: {}'.format(train_data['prompt'][20]))
print('Sample answer: {}'.format(train_data['utterance'][20]))

Sample context: grateful
Sample question: i have always been loyal to my wife .
Sample answer: what do you mean it has not been easy ? how close have you come to cheating ?


In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [2]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [3]:
from transformers import TFAutoModelForQuestionAnswering

db_model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForQuestionAnswering: ['vocab_layer_norm', 'vocab_transform', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
def preprocess_function(data):
    questions = [q.strip() for q in data["prompt"]]
    context = data['utterance']

    inputs = tokenizer(
        questions,
        context,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    # print("offset_mapping", offset_mapping)
    answers = data["utterance"]
    print("answers", answers)
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer.index(answer[0])
        end_char = answer.index(answer[len(answer[0])-1])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [34]:
train_data.head()

,Unnamed: 0,context,prompt,utterance
0,0,disappointed,i remember going to the fireworks with my best...,i remember going to see the fireworks with my ...
1,1,disappointed,i remember going to the fireworks with my best...,was this a friend you were in love with comma ...
2,2,disappointed,i remember going to the fireworks with my best...,this was a best friend . i miss her .
3,3,disappointed,i remember going to the fireworks with my best...,where has she gone ?
4,4,disappointed,i remember going to the fireworks with my best...,we no longer talk .


In [16]:
!pip install --quiet datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 17.5 MB/s eta 0:00:00


In [40]:

from datasets import Dataset, DatasetDict

# assume df is the pandas dataframe you want to convert
train_dataset = Dataset.from_pandas(train_data[:1000])
test_dataset = Dataset.from_pandas(test_data[:200])
# create a DatasetDict object with a key "my_dataset"
dataset_dict = DatasetDict({"tr_dataset": train_dataset,"ts_dataset": test_dataset})

dataset_dict 

DatasetDict({
    tr_dataset: Dataset({
        features: ['Unnamed: 0', 'context', 'prompt', 'utterance'],
        num_rows: 1000
    })
    ts_dataset: Dataset({
        features: ['Unnamed: 0', 'context', 'prompt', 'utterance'],
        num_rows: 200
    })
})

In [30]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [41]:

tokenized_ds = dataset_dict.map(preprocess_function, batched=True, remove_columns=dataset_dict["tr_dataset"].column_names)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

answers ['i remember going to see the fireworks with my best friend . it was the first time we ever spent time alone together . although there was a lot of people comma we felt like the only people in the world .', 'was this a friend you were in love with comma or just a best friend ?', 'this was a best friend . i miss her .', 'where has she gone ?', 'we no longer talk .', 'oh was this something that happened because of an argument ?', 'it feels like hitting to blank wall when i see the darkness', 'oh ya ? i do not really see how', 'dont you feel so . . its a wonder', 'i do actually hit blank walls a lot of times but i get by', 'i virtually thought so . . and i used to get sweatings', 'wait what are sweatings', 'hi how are you doing today', 'doing good . . how about you', 'im good comma trying to understand how someone can feel like hitting a blank wall when they see the darkness', 'it is quite strange that you didnt imagine it', 'i dont imagine feeling a lot comma maybe your on to som

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

answers ['yeah about years ago i had a horrifying experience . it was their fault but they hit the water barrels and survived . they had no injuries but they almost ran me off the road .', 'did you suffer any injuries ?', 'no i was not hit . it turned out they were drunk . i felt guilty but realized it was his fault .', 'why did you feel guilty ? people really should not drive drunk .', 'i do not know i was new to driving and had not experienced anything like that . i felt like my horn made him swerve into the water barrels .', 'well comma can you tell me about your experience ? i think we swapped places', 'yeah i wanted to tell you about the time i was hit by a drunk driver im so happy to still be alive after that experience', 'oh my goodness comma that is very scary ! i hope you are okay now and the drunk driver was punished for his actions ?', 'yeah he was punished hes in jail still', 'i there comma dont know what to do comma jst broke up with my girlfirned comma we were years toget

In [42]:
tokenized_ds

DatasetDict({
    tr_dataset: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 1000
    })
    ts_dataset: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 200
    })
})

In [50]:
from transformers import create_optimizer

batch_size = 8
num_epochs = 2
total_train_steps = (len(tokenized_ds["tr_dataset"]) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
)

### Load Pretrained_BERT

In [26]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForQuestionAnswering: ['vocab_layer_norm', 'vocab_transform', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_59', 'qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [48]:
!pip install datasets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [49]:
!pip show datasets

Name: datasets
Version: 2.10.1
Summary: HuggingFace community-driven open-source library of datasets
Home-page: https://github.com/huggingface/datasets
Author: HuggingFace Inc.
Author-email: thomas@huggingface.co
License: Apache 2.0
Location: /usr/local/lib/python3.9/dist-packages
Requires: aiohttp, dill, fsspec, huggingface-hub, multiprocess, numpy, packaging, pandas, pyarrow, pyyaml, requests, responses, tqdm, xxhash
Required-by: 


In [57]:
columns = ['input_ids',  'attention_mask', 'start_positions', 'end_positions']

encoded_tf_dataset = tokenized_ds["tr_dataset"].to_tf_dataset(
    columns=columns,
   shuffle=True,
    batch_size=8,
    collate_fn=data_collator
)

encoded_tf_dataset2 = tokenized_ds["ts_dataset"].to_tf_dataset(
    columns=columns,
   shuffle=True,
    batch_size=8,
    collate_fn=data_collator
)

In [58]:
#prepare dataset
# tf_train_ds = model.prepare_tf_dataset(
#     tokenized_ds["tr_dataset"],
#     shuffle=True,
#     batch_size=8,
#     collate_fn=data_collator,
# )

# tf_validation_ds = model.prepare_tf_dataset(
#     tokenized_ds["ts_dataset"],
#     shuffle=False,
#     batch_size=8,
#     collate_fn=data_collator,
# )

In [59]:
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [60]:
model.fit(x=encoded_tf_dataset, validation_data=encoded_tf_dataset2, epochs=1)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


125/125 [==============================] - 76s 413ms/step - loss: 1.8938 - val_loss: 0.1401


In [69]:
# Save the distilBert model as a SavedModel.
save_directory = "/content/drive/MyDrive/code/saved_dbert_models" 
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)


('/content/drive/MyDrive/code/saved_dbert_models/tokenizer_config.json',
 '/content/drive/MyDrive/code/saved_dbert_models/special_tokens_map.json',
 '/content/drive/MyDrive/code/saved_dbert_models/vocab.txt',
 '/content/drive/MyDrive/code/saved_dbert_models/added_tokens.json',
 '/content/drive/MyDrive/code/saved_dbert_models/tokenizer.json')

In [61]:
question = "i have always been loyal to my wife ."
context = "what do you mean it has not been easy ? how close have you come to cheating ?"

In [62]:
# Sample context: grateful
# Sample question: i have always been loyal to my wife .
# Sample answer: what do you mean it has not been easy ? how close have you come to cheating ?

In [72]:
from transformers import pipeline

#load saved model to use
new_tokenizer = AutoTokenizer.from_pretrained(save_directory)
new_model = TFAutoModelForQuestionAnswering.from_pretrained(save_directory)

question_answerer = pipeline("question-answering", model=new_model, tokenizer=new_tokenizer)
question_answerer(question=question, context=context)

Some layers from the model checkpoint at /content/drive/MyDrive/code/saved_dbert_models were not used when initializing TFDistilBertForQuestionAnswering: ['dropout_59']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at /content/drive/MyDrive/code/saved_dbert_models and are newly initialized: ['dropout_117']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'score': 0.9902945756912231, 'start': 0, 'end': 4, 'answer': 'what'}